In [49]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec, KeyedVectors 
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
stop_words = set(stopwords.words("english"))
import networkx as nx
from rouge_score import rouge_scorer

[nltk_data] Downloading package punkt to C:\Users\Malathi
[nltk_data]     M\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Malathi
[nltk_data]     M\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

import joblib


In [ ]:
#pip install pandas numpy scikit-learn nltk rouge-score torch transformers sentencepiece tensorflow


In [50]:
df = pd.read_csv("C:/Users/Malathi M/OneDrive/Documents/MDTE25/guvi final project/Main project/news.tsv.zip", sep="\t")
print(df.head())

  News ID Category         Topic  \
0  N10000   sports        soccer   
1  N10001     news  newspolitics   
2  N10002     news        newsus   
3  N10003     news  newspolitics   
4  N10004     news     newsworld   

                                            Headline  \
0  Predicting Atlanta United's lineup against Col...   
1  Mitch McConnell: DC statehood push is 'full bo...   
2            Home In North Highlands Damaged By Fire   
3  Meghan McCain blames 'liberal media' and 'thir...   
4                            Today in History: Aug 1   

                                           News body  \
0  Only FIVE internationals allowed, count em, FI...   
1  WASHINGTON -- Senate Majority Leader Mitch McC...   
2  NORTH HIGHLANDS (CBS13)   Fire damaged a home ...   
3  Meghan McCain is speaking out after a journali...   
4  1714: George I becomes King Georg Ludwig, Elec...   

                                Title entity  \
0  {"Atlanta United's": 'Atlanta United FC'}   
1            

In [52]:
df = df.dropna(subset=["Headline", "News body", "Category"])

df["text"] = df["Headline"].astype(str) + " " + df["News body"].astype(str)
df["summary"] = df["Headline"].astype(str)

df = df.rename(columns={"Category": "label"})

# Use only 50%
df = df.sample(frac=0.5, random_state=42).reset_index(drop=True)

df["article"] = df["text"]
df.head()


,News ID,label,Topic,Headline,News body,Title entity,Entity content,text,summary,article
0,N95333,news,newsus,This dog's smile will melt your heart,"Mocca lives in Yokohama, Japan, and is a Shiba...",{},{},This dog's smile will melt your heart Mocca li...,This dog's smile will melt your heart,This dog's smile will melt your heart Mocca li...
1,N41910,lifestyle,shop-all,The Most Popular Walmart Item in Every State,What are the most oft-ordered Walmart products...,{'Walmart': 'Walmart'},"{'Walmart': {'type': 'item', 'id': 'Q18615334'...",The Most Popular Walmart Item in Every State W...,The Most Popular Walmart Item in Every State,The Most Popular Walmart Item in Every State W...
2,N88506,finance,finance-real-estate,Photos: Look Glenn Close's 'Beanfield' estate ...,Emmy winning actress Glen Close listed her Bed...,"{""Glenn Close's"": 'Glenn Close', 'Bedford': 'B...","{'Glenn Close': {'type': 'item', 'id': 'Q37231...",Photos: Look Glenn Close's 'Beanfield' estate ...,Photos: Look Glenn Close's 'Beanfield' estate ...,Photos: Look Glenn Close's 'Beanfield' estate ...
3,N114168,news,newscrime,Hillsborough Sheriff's Office sweep results in...,TAMPA More than 80 people have been arrested...,{'human trafficking': 'Human trafficking'},"{'Human trafficking': {'type': 'item', 'id': '...",Hillsborough Sheriff's Office sweep results in...,Hillsborough Sheriff's Office sweep results in...,Hillsborough Sheriff's Office sweep results in...
4,N35279,video,peopleandplaces,Family of missing Connecticut mom blast 'Gone ...,Family members and friends of Jennifer Dulos s...,{'Connecticut': 'Connecticut'},"{'Connecticut': {'type': 'item', 'id': 'Q58425...",Family of missing Connecticut mom blast 'Gone ...,Family of missing Connecticut mom blast 'Gone ...,Family of missing Connecticut mom blast 'Gone ...


 Text Cleaning

In [53]:
import re
import string
def clean_text(text):
    if not isinstance(text, str):
        return ""

    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"http\S+|www\S+|https\S+", " ", text)
    text = text.encode("ascii", "ignore").decode()
    text = text.translate(str.maketrans(string.punctuation, " "*len(string.punctuation)))
    text = re.sub(r"[^a-zA-Z0-9 ]", " ", text)
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_article"] = df["article"].apply(clean_text)
df["clean_summary"] = df["summary"].apply(clean_text)


TF-IDF Summarizer

In [54]:
def tfidf_summarize(text, top_n=3):
    cleaned = clean_text(text)
    sentences = sent_tokenize(cleaned)

    if len(sentences) <= top_n:
        return " ".join(sentences)

    vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = vectorizer.fit_transform(sentences)

    scores = tfidf_matrix.mean(axis=1).A.flatten()
    ranked_idx = np.argsort(scores)[::-1][:top_n]

    selected = sorted(ranked_idx)
    return " ".join([sentences[i] for i in selected])


TextRank Summarizer

In [55]:
def textrank_summarize(text, top_n=3):
    cleaned = clean_text(text)
    sentences = sent_tokenize(cleaned)

    if len(sentences) <= top_n:
        return " ".join(sentences)

    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(sentences).toarray()

    sim_matrix = cosine_similarity(vectors)
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)

    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    selected = sorted([idx for idx, _ in ranked[:top_n]])

    return " ".join([sentences[i] for i in selected])


Seq2Seq (LSTM) – TINY MODEL

In [61]:
tokenizer = Tokenizer(num_words=3000, oov_token="<OOV>")
tokenizer.fit_on_texts(df["article"])

X = tokenizer.texts_to_sequences(df["article"])
y = tokenizer.texts_to_sequences(df["summary"])

max_len = 80
X = pad_sequences(X, maxlen=max_len, padding="post")
y = pad_sequences(y, maxlen=max_len, padding="post")


In [62]:
vocab_size = 3000

enc_inputs = Input(shape=(max_len,))
enc_emb = Embedding(vocab_size, 32)(enc_inputs)
enc_lstm = LSTM(32, return_state=True)
_, h, c = enc_lstm(enc_emb)

dec_inputs = Input(shape=(max_len,))
dec_emb = Embedding(vocab_size, 32)(dec_inputs)
dec_lstm = LSTM(32, return_sequences=True)
dec_out = dec_lstm(dec_emb, initial_state=[h, c])
dec_dense = Dense(vocab_size, activation="softmax")
outputs = dec_dense(dec_out)

seq2seq_model = Model([enc_inputs, dec_inputs], outputs)
seq2seq_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
seq2seq_model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 80)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 80)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 80, 32)    │     96,000 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, 80, 32)    │     96,000 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, 32),      │      8,320 │ embedding_4[0][0] │
│                     │ (None, 32),       │            │                   │
│                     │ (None, 32)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ (None, 80, 32)    │      8,320 │ embedding_5[0][0… │
│                     │                   │            │ lstm_4[0][1],     │
│                     │                   │            │ lstm_4[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 80, 3000)  │     99,000 │ lstm_5[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 307,640 (1.17 MB)

 Trainable params: 307,640 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

In [63]:
seq2seq_model.fit(
    [X, y],
    y,
    epochs=3,
    batch_size=16
)


Epoch 1/3
3554/3554 ━━━━━━━━━━━━━━━━━━━━ 559s 156ms/step - loss: 0.6756
Epoch 2/3
3554/3554 ━━━━━━━━━━━━━━━━━━━━ 560s 158ms/step - loss: 0.2508
Epoch 3/3
3554/3554 ━━━━━━━━━━━━━━━━━━━━ 550s 155ms/step - loss: 0.0914


In [64]:
def seq2seq_summarize(text, max_len=20):
    seq = tokenizer.texts_to_sequences([text])
    seq = pad_sequences(seq, maxlen=80, padding="post")
    preds = seq2seq_model.predict([seq, seq], verbose=0)
    pred_ids = np.argmax(preds[0], axis=1)
    return " ".join(tokenizer.index_word.get(i, "") for i in pred_ids if i != 0)


In [65]:
scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeL"],
    use_stemmer=True
)

def evaluate(model_fn, df, n=50):
    r1, r2, rL = [], [], []

    for i in range(min(n, len(df))):
        ref = df.iloc[i]["summary"]
        pred = model_fn(df.iloc[i]["article"])
        score = scorer.score(ref, pred)
        r1.append(score["rouge1"].fmeasure)
        r2.append(score["rouge2"].fmeasure)
        rL.append(score["rougeL"].fmeasure)

    return np.mean(r1), np.mean(r2), np.mean(rL)


In [66]:
results = []

results.append(("TF-IDF", *evaluate(tfidf_summarize, df)))
results.append(("TextRank", *evaluate(textrank_summarize, df)))
results.append(("Seq2Seq", *evaluate(seq2seq_summarize, df)))

results_df = pd.DataFrame(
    results,
    columns=["Model", "ROUGE-1", "ROUGE-2", "ROUGE-L"]
)

results_df["Average"] = results_df[["ROUGE-1", "ROUGE-2", "ROUGE-L"]].mean(axis=1)
results_df


,Model,ROUGE-1,ROUGE-2,ROUGE-L,Average
0,TF-IDF,0.159553,0.148539,0.159553,0.155882
1,TextRank,0.159553,0.148539,0.159553,0.155882
2,Seq2Seq,0.124501,0.057310,0.112941,0.098250


final

In [67]:
best_model = results_df.sort_values("Average", ascending=False).iloc[0]["Model"]

config = {
    "best_model": best_model,
    "top_k": 3
}

joblib.dump(config, "best_from_scratch_summarizer.pkl")
results_df.to_csv("from_scratch_summarization_results.csv", index=False)

print("Best model:", best_model)


Best model: TF-IDF


In [ ]:
scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeL"],
    use_stemmer=True
)

def evaluate_model(summarizer_fn, df, samples=50):
    r1, r2, rL = [], [], []

    for i in range(min(samples, len(df))):
        text = df.iloc[i]["article"]
        ref = df.iloc[i]["summary"]
        pred = summarizer_fn(text)

        scores = scorer.score(ref, pred)
        r1.append(scores["rouge1"].fmeasure)
        r2.append(scores["rouge2"].fmeasure)
        rL.append(scores["rougeL"].fmeasure)

    return {
        "rouge1": np.mean(r1),
        "rouge2": np.mean(r2),
        "rougeL": np.mean(rL)
    }


In [ ]:
tfidf_scores = evaluate_model(tfidf_summarize, df)
textrank_scores = evaluate_model(textrank_summarize, df)

results = pd.DataFrame([
    {"Model": "TF-IDF", **tfidf_scores},
    {"Model": "TextRank", **textrank_scores}
])

results["Average"] = results[["rouge1", "rouge2", "rougeL"]].mean(axis=1)
results.round(4)


In [ ]:
results.to_csv("rouge_eval_results.csv", index=False)


In [ ]:
best_model_name = results.sort_values("Average", ascending=False).iloc[0]["Model"]

best_model = {
    "name": best_model_name,
    "top_n": 3
}

import joblib
joblib.dump(best_model, "best_summarizer.pkl")


In [3]:
df = df.dropna(subset=['Headline', 'News body'])
df['Headline'] = df['Headline'].astype(str)
df['News body'] = df['News body'].astype(str)


In [4]:
df.isnull().sum()

News ID           0
Category          0
Topic             0
Headline          0
News body         0
Title entity      0
Entity content    0
dtype: int64

In [5]:
# Combine headline and body
df["text"] = df["Headline"].fillna("") + " " + df["News body"].fillna("")

# Rename label column
df = df.rename(columns={"Category": "label"})

# Drop rows missing essential fields
df = df.dropna(subset=["label", "text"])

In [6]:
# Take only 50% of the data to reduce computation
df = df.sample(frac=0.5, random_state=42).reset_index(drop=True)

In [7]:
# This is required for evaluation
df["full_text"] = df["text"] 

In [8]:
# Input text
df["article"] = df["full_text"]

# Ground-truth summary
df["summary"] = df["Headline"]

df = df.dropna(subset=["article", "summary"])

In [9]:
import string
import re
def clean_text(text):

    if pd.isna(text):
        return ""
    
    text = str(text)

    # Remove HTML tags
    text = re.sub(r'<.*?>', ' ', text)

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', ' ', text)

    # Remove emojis
    text = text.encode("ascii", "ignore").decode()

    # Replace punctuation with space
    text = text.translate(str.maketrans(string.punctuation, " "*len(string.punctuation)))

    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9 ]', ' ', text)

    # Convert to lowercase
    text = text.lower()

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [10]:
df["clean_text"] = df["text"].apply(clean_text)
df["clean_text"].head()

0    this dog s smile will melt your heart mocca li...
1    the most popular walmart item in every state w...
2    photos look glenn close s beanfield estate in ...
3    hillsborough sheriff s office sweep results in...
4    family of missing connecticut mom blast gone g...
Name: clean_text, dtype: object

In [11]:
df["clean_article"] = df["article"].apply(clean_text)
df["clean_summary"] = df["summary"].apply(clean_text)


sentence tokanization

In [12]:
def sentence_split(text):
    return sent_tokenize(text)

df['sentences'] = df['clean_article'].apply(sentence_split)
df[['sentences']].head()


,sentences
0,[this dog s smile will melt your heart mocca l...
1,[the most popular walmart item in every state ...
2,[photos look glenn close s beanfield estate in...
3,[hillsborough sheriff s office sweep results i...
4,[family of missing connecticut mom blast gone ...


In [ ]:
#!pip install rouge-score


In [39]:

from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx


In [40]:

# TEXT RANK SUMMARIZER

def textrank_summarize(text, top_n=3):
    cleaned = clean_text(text)
    sentences = sent_tokenize(cleaned)

    if len(sentences) <= top_n:
        return " ".join(sentences)

    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(sentences).toarray()

    sim_matrix = cosine_similarity(vectors)

    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)

    ranked = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    summary = " ".join([s for _, s in ranked[:top_n]])

    return summary

In [41]:

# TF-IDF SENTENCE SCORING

def tfidf_summarize(text, top_n=3):
    cleaned = clean_text(text)
    sentences = sent_tokenize(cleaned)

    if len(sentences) <= top_n:
        return " ".join(sentences)

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)

    scores = tfidf_matrix.mean(axis=1).A.flatten()

    ranked_idx = np.argsort(scores)[::-1]
    selected = [sentences[i] for i in ranked_idx[:top_n]]

    return " ".join(selected)


# REFERENCE SUMMARY (WEAK BASELINE)

def reference_summary(text):
    sents = sent_tokenize(clean_text(text))
    return " ".join(sents[:2])

In [ ]:
from rouge_score import rouge_scorer
import numpy as np

In [42]:

# ROUGE EVALUATION

scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

def evaluate_model(summarizer_fn, df, samples=50):

    rouge1_scores, rouge2_scores, rougeL_scores = [], [], []

    for i in range(min(samples, len(df))):

        text = df.iloc[i]["full_text"]
        ref = reference_summary(text)
        pred = summarizer_fn(text)

        scores = scorer.score(ref, pred)

        rouge1_scores.append(scores["rouge1"].fmeasure)
        rouge2_scores.append(scores["rouge2"].fmeasure)
        rougeL_scores.append(scores["rougeL"].fmeasure)

    return {
        "rouge1": np.mean(rouge1_scores),
        "rouge2": np.mean(rouge2_scores),
        "rougeL": np.mean(rougeL_scores),
    }

In [44]:

# RUN EVALUATION

print("Evaluating TextRank")
textrank_scores = evaluate_model(textrank_summarize, df)

print("Evaluating TF-IDF")
tfidf_scores = evaluate_model(tfidf_summarize, df)


Evaluating TextRank
Evaluating TF-IDF


In [45]:
# SAVE RESULTS TO CSV

OUTPUT_CSV = "rouge_eval_results.csv"  # change if needed

new_results = pd.DataFrame([
    {
        "Model": "TextRank",
        "rouge1": textrank_scores["rouge1"],
        "rouge2": textrank_scores["rouge2"],
        "rougeL": textrank_scores["rougeL"],
        "rougeLsum": textrank_scores["rougeL"],
        "Average Score": np.mean([
            textrank_scores["rouge1"],
            textrank_scores["rouge2"],
            textrank_scores["rougeL"],
        ])
    },
    {
        "Model": "TF-IDF",
        "rouge1": tfidf_scores["rouge1"],
        "rouge2": tfidf_scores["rouge2"],
        "rougeL": tfidf_scores["rougeL"],
        "rougeLsum": tfidf_scores["rougeL"],
        "Average Score": np.mean([
            tfidf_scores["rouge1"],
            tfidf_scores["rouge2"],
            tfidf_scores["rougeL"],
        ])
    }
])
new_results = new_results.round(4)

# If CSV exists → append, else create new
try:
    old_df = pd.read_csv(OUTPUT_CSV)
    final_df = pd.concat([old_df, new_results], ignore_index=True)
except FileNotFoundError:
    final_df = new_results

final_df.to_csv(OUTPUT_CSV, index=False)

print("\nEvaluation Completed.\nSaved to:", OUTPUT_CSV)


Evaluation Completed.
Saved to: rouge_eval_results.csv


In [46]:
import pandas as pd
df1=pd.read_csv("C:/Users/Malathi M/OneDrive/Documents/MDTE25/guvi final project/Main project/summarization/rouge_eval_results.csv")
df1

,Model,rouge1,rouge2,rougeL,rougeLsum,Average Score
0,TextRank,1.0,1.0,1.0,1.0,1.0
1,TF-IDF,1.0,1.0,1.0,1.0,1.0
2,TextRank,1.0,1.0,1.0,1.0,1.0
3,TF-IDF,1.0,1.0,1.0,1.0,1.0


In [24]:
sample_text = df['full_text'][0]
sam = textrank_summarize(sample_text)
print(sam)

this dog s smile will melt your heart mocca lives in yokohama japan and is a shiba inu with a unique smile when greeting someone this dog can t hide its happiness eyes partly closed smile wide ears going horizontal plus a tail wagging like a helicopter say it all


In [ ]:
# STEP 1: CREATE SUMMARIZER FUNCTIONS FILE

summarizer_code = """
import re
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt', quiet=True)

def clean_text(text):
    # Handle empty or NaN-like values
    if not isinstance(text, str):
        return ""
    
    import re
    # Remove HTML tags
    text = re.sub(r"<.*?>", " ", text)
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", " ", text)
    # Keep only letters, numbers, basic punctuation
    text = re.sub(r"[^a-zA-Z0-9\s.,!?]", " ", text)
    # Lowercase text
    text = text.lower()
    # Remove extra whitespace
    text = " ".join(text.split())
    
    return text


def tfidf_summarize(text, top_n=3):
    cleaned = clean_text(text)
    sentences = sent_tokenize(cleaned)
    if len(sentences) <= top_n:
        return " ".join(sentences)

    vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = vectorizer.fit_transform(sentences)
    scores = tfidf_matrix.sum(axis=1).A.flatten()

    ranked_idx = sorted(scores.argsort()[::-1][:top_n])
    return " ".join([sentences[i] for i in ranked_idx])

def summarize(text, model_name="TF-IDF", top_n=3):
    if model_name == "TF-IDF":
        return tfidf_summarize(text, top_n)
    else:
        raise ValueError("Unknown summarization model")
"""

with open("../Summarization/summarizer_functions.py", "w", encoding="utf-8") as f:
    f.write(summarizer_code)

print("✅ summarizer_functions.py saved")


✅ summarizer_functions.py saved


In [37]:
import joblib

best_model = {
    "name": "TF-IDF",
    "top_n": 3
}

joblib.dump(best_model, "../Summarization/best_summarizer_TF-IDF.pkl")

print("✅ Best summarizer saved")


✅ Best summarizer saved


In [38]:
from summarizer_functions import summarize

best_model = joblib.load("../Summarization/best_summarizer_TF-IDF.pkl")

text = df.loc[0, "News body"]

summary = summarize(
    text,
    model_name=best_model["name"],
    top_n=best_model["top_n"]
)

print("SUMMARY:\n", summary)


NameError: name 'pd' is not defined

In [ ]:
best_model = {
    "name": best_model_name,
    "top_n": 3
}

import joblib
joblib.dump(best_model, f"../Summarization/best_summarizer_{best_model_name}.pkl")

summarizer_code = """
import re
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

# ---------------------------------------------------------
# CLEANING FUNCTION
# ---------------------------------------------------------
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"http\\S+|www\\S+|https\\S+", " ", text)
    text = re.sub("[" 
                  u"\\U0001F600-\\U0001F64F"
                  u"\\U0001F300-\\U0001F5FF"
                  u"\\U0001F680-\\U0001F6FF"
                  u"\\U0001F1E0-\\U0001F1FF"
                  "]+", "", text)
    text = re.sub(r"[^a-zA-Z0-9\\s.,!?]", " ", text)
    text = text.lower()
    text = " ".join(text.split())
    return text
    
# ---------------------------------------------------------
# TEXT RANK SUMMARY
# ---------------------------------------------------------
def textrank_summarize(text, top_n=3):
    cleaned = clean_text(text)
    sentences = sent_tokenize(cleaned)
    if len(sentences) <= top_n:
        return " ".join(sentences)
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(sentences).toarray()
    sim_matrix = cosine_similarity(vectors)
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)
    ranked = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    return " ".join([s for _, s in ranked[:top_n]])
    # ---------------------------------------------------------
# TF-IDF SCORING SUMMARY
# ---------------------------------------------------------
def tfidf_summarize(text, top_n=3):
    cleaned = clean_text(text)
    sentences = sent_tokenize(cleaned)
    if len(sentences) <= top_n:
        return " ".join(sentences)
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)
    scores = tfidf_matrix.mean(axis=1).A.flatten()
    ranked_idx = scores.argsort()[::-1]
    selected = [sentences[i] for i in ranked_idx[:top_n]]
    return " ".join(selected)
    
# ---------------------------------------------------------
# UNIFIED SUMMARIZER FUNCTION
# ---------------------------------------------------------
def summarize(text, model_name, top_n=3):
    if model_name == "TextRank":
        return textrank_summarize(text, top_n)
    elif model_name == "TF-IDF":
        return tfidf_summarize(text, top_n)
    else:
        raise ValueError("Unknown summarization model: " + model_name)

"""

with open("summarizer_functions.py", "w", encoding="utf-8") as f:
    f.write(summarizer_code)

print("Saved summarizer functions → ../Summarization/summarizer_functions.py")